In [1]:
import numpy as np
import pandas as pd
import time
import os
import pathlib
import copy

import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras import losses, optimizers
from tensorflow.keras.models import Sequential, load_model
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Input, Activation, Dense, TimeDistributed,Conv2D,MaxPooling2D, Lambda,UpSampling2D
from tensorflow.keras.layers import add, dot, concatenate, LSTM, Bidirectional,Reshape,Flatten
import tensorflow.keras as keras
from tensorflow.compat.v1.keras.layers import CuDNNLSTM, CuDNNGRU
#from tf.compat.v1.keras.layers.CuDNNLSTM
from Utils.PlotLosses import PlotLosses
from Utils.Utils import dist_coordinates, TrainModel, s_DataFrame, s_relative_and_std, cal_accuracy, euclidean_distance_loss,set_input,s_labels, change_relative_coordinates, get_reculsive_error, re_coordinates
from Utils.Utils import get_reculsive_error_m3_diff_timestep, get_reculsive_error_m123, get_reculsive_error_m3_only_nearest,cal_accuracy_tmp, dist_euclidean
path = './Data_v2.0/7Robots/TestSet'
data_test = pd.read_csv(path+'/Test.csv')

std_coordinates = 500
std_orientation = 180

robots = ['H','F1','F2','F3','F4','F5','T']
std_robot = 'T'
coordinates = ['x','y']
orientation = ['o']
_n_hist = 10
_n_samples = 26

In [2]:
def get_reculsive_error(_Model='',timedifference=[],timeshift=0,_n_model = 1,start=0,std_robot='T',_obs_list=[],pred_list=[],robots=['H','F1','F2','F3','F4','F5','T'],data_test = '',_n_samples=12,_n_hist=10,std_coordinates=500) :
    other_robots = copy.deepcopy(robots)
 #   print(other_robots)
#    print(std_robot)
    other_robots.remove(std_robot)
    pred_result = pd.DataFrame()
    error_model1=list()
        
    test_data = s_relative_and_std(data = data_test, std_coordinates = std_coordinates,n_time_in_instance=_n_samples,std_robot =std_robot,other_robots=other_robots)
    _answer = s_relative_and_std(data = data_test, std_coordinates = std_coordinates,n_time_in_instance=_n_samples,std_robot =std_robot,other_robots=other_robots)

    for i in range(0,len(_obs_list)) :
        _errors = list()
        pred_result = pd.DataFrame()

        for step in range(0,_n_samples-_n_hist-1-i-timeshift) :
            
            ##Set input
            if timeshift == 0 :
                x_test = set_input(_obs_list = _obs_list[i],
                                   start=start,pred_list= pred_list[i],
                                   step = step, _hist_len = _n_hist,
                                   _n_samples = _n_samples, 
                                   data1 = test_data,data2 = test_data)    
            else :
                x_test = set_input(_obs_list = _obs_list[i], 
                                   start=start, pred_list= pred_list[i],
                                   timedifference = timedifference,
                                   step = step,  _hist_len = _n_hist,
                                   _n_samples = _n_samples, data1 = test_data,data2 = test_data)    
            
            ## Predict
            _result = _Model.predict(x_test)

            ## Insert pred_result to next input
            if _n_model == 1 :
                _label_list = [['{}_{}_x'.format(step+_n_hist,robots[4-i]),'{}_{}_y'.format(step+_n_hist,robots[4-i])],
                        ['{}_{}_o'.format(step+_n_hist,robots[5-i]),'{}_{}_o'.format(step+_n_hist,robots[4-i]),'{}_{}_o'.format(step+_n_hist+1,robots[4-i])]]
            elif _n_model == 2 :
                _label_list = [['{}_{}_x'.format(step+_n_hist,robots[3-i]),'{}_{}_y'.format(step+_n_hist,robots[3-i])],
                               ['{}_{}_o'.format(step+_n_hist,robots[3-i]),'{}_{}_o'.format(step+_n_hist,robots[4-i]),'{}_{}_o'.format(step+_n_hist,robots[5-i]),'{}_{}_o'.format(step+_n_hist+1,robots[5-i])]]
            else :
                _label_list = [['{}_{}_x'.format(step+_n_hist,robots[2-i]),'{}_{}_y'.format(step+_n_hist,robots[2-i])],
                               ['{}_{}_o'.format(step+_n_hist,robots[2-i]),'{}_{}_o'.format(step+_n_hist,robots[3-i]),'{}_{}_o'.format(step+_n_hist,robots[4-i]),'{}_{}_o'.format(step+_n_hist,robots[5-i]),'{}_{}_o'.format(step+_n_hist+1,robots[5-i])]]

#                               ['{}_{}_o'.format(step+_n_hist,robots[2-i]),'{}_{}_o'.format(step+_n_hist,robots[3-i])]]              
            print('Label : {}'.format(_label_list))

#            print(_label_list)
#            print(pred_result.columns())
            for j in range(len(_label_list)) :
                for k in range(len(_label_list[j])) :
                    pred_result[_label_list[j][k]] = _result[j][:,k]
                   
            ## concatenates pred_result and original data
            for columns in pred_result.columns :
                test_data[columns] = pred_result[columns]

        ## Change std_robot and change relative coordinates
        _std_robot = robots[5-i]
        other_robots.remove(_std_robot)
#        print(_std_robot)
#        print(other_robots)
        test_data = change_relative_coordinates(std_robot=_std_robot,other_robots=other_robots,data = test_data,n_samples=_n_samples)
        _answer = change_relative_coordinates(std_robot=_std_robot,other_robots=other_robots,data = _answer,n_samples=_n_samples)
       
    return test_data


In [3]:
K.clear_session()

In [4]:
test_data = s_relative_and_std(data = data_test, std_coordinates = std_coordinates,n_time_in_instance=_n_samples,std_robot ='T',other_robots=['H','F1','F3','F2','F4','F5'])

## Load model
* Model 1 : ICRA
* Model 2 : 2 cues
* Model 3 : 3 cues (std - ICRA)
* Model 3-2 : 1 cues nearest from the target predicted
* Model 3-3 : 3 cues (diffrent time step)

In [16]:
_model1 = keras.models.load_model('./SaveModels/Modified_Models/Model3_sametime_1.h', custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
print(_model1.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 12)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 12)]         0                                            
__________________________________________________________________________________________________
History_Hidden1 (Bidirectional) (None, 320)          221440      input_1[0][0]                    
__________________________________________________________________________________________________
Observer_Hidden1 (Dense)        (None, 80)           1040        input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
Model1 = list()
for i in range(0,5) :
    _model = keras.models.load_model('./SaveModels/Modified_Models/Model1_plusOrientation_{}.h'.format(i), custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
    Model1.append(_model)

In [ ]:
Model1 = list()
for i in range(0,5) :
    _model = keras.models.load_model('./SaveModels/Modified_Models/Model1_{}.h'.format(i+5), custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
    Model1.append(_model)

In [ ]:
Model1_1 = list()
for i in range(0,5) :
    _model = keras.models.load_model('./SaveModels/Modified_Models/Model1_plusOrientation_{}.h'.format(i), custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
    Model1_1.append(_model)

In [5]:
Model2 = list()
for i in range(0,5) :
    _model = keras.models.load_model('./SaveModels/Modified_Models/Model2_{}.h'.format(i+5), custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
    Model2.append(_model)

In [ ]:
Model2 = list()
for i in range(0,5) :
    _model = keras.models.load_model('./SaveModels/Modified_Models/Model2_sametime_{}.h'.format(i), custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
    Model2.append(_model)

In [ ]:
Model3 = list()
for i in range(0,5) :
    _model = keras.models.load_model('./SaveModels/Modified_Models/Model3_difftime_{}.h'.format(i+5), custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
#    _model = keras.models.load_model('./SaveModels/Modified_Models/Model3_sametime_{}.h'.format(i+5), custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
    Model3.append(_model)

In [5]:
Model3_2 = list()
for i in range(0,5) :
    _model = keras.models.load_model('./SaveModels/Modified_Models/Model3_nearest_only_{}.h'.format(i+5), custom_objects={'euclidean_distance_loss': euclidean_distance_loss})
    Model3_2.append(_model)

In [ ]:
## concatenates pred_result and original data
## Change std_robot and change relative coordinates 
## calculate error

## Reculsive Error

### Model1

In [ ]:
#_obs_list = [['_F3_x','_F3_y','_T_o'],['_F2_x','_F2_y','_F3_o'],['_F1_x','_F1_y','_F2_o']]
#pred_list = [['_F2_x','_F2_y','_F3_o','_F2_o'],['_F1_x','_F1_y','_F2_o','_F1_o'],['_H_x','_H_y','_F1_o','_H_o']]
#robots = ['H','F1','F2','F3','T']
std_coordinates = 260
std_orientation = 180

_obs_list = [['_F5_x','_F5_y'],['_F4_x','_F4_y'],
             ['_F3_x','_F3_y'],['_F2_x','_F2_y'],
             ['_F1_x','_F1_y']]
pred_list = [['_F4_x','_F4_y','_F5_o','_F4_o'],['_F3_x','_F3_y','_F4_o','_F3_o'],
            ['_F2_x','_F2_y','_F3_o','_F2_o'],['_F1_x','_F1_y','_F2_o','_F1_o'],
            ['_H_x','_H_y','_F1_o','_H_o']]
robots = ['H','F1','F2','F3','F4','F5','T']

In [ ]:
## Orientation 추가 모델

std_coordinates = 260
std_orientation = 180

_obs_list = [['_F5_x','_F5_y','_T_o'],['_F4_x','_F4_y','_F5_o'],
             ['_F3_x','_F3_y','_F4_o'],['_F2_x','_F2_y','_F3_o'],
             ['_F1_x','_F1_y','_F2_o']]
pred_list = [['_F4_x','_F4_y','_F5_o','_F4_o'],['_F3_x','_F3_y','_F4_o','_F3_o'],
            ['_F2_x','_F2_y','_F3_o','_F2_o'],['_F1_x','_F1_y','_F2_o','_F1_o'],
            ['_H_x','_H_y','_F1_o','_H_o']]
robots = ['H','F1','F2','F3','F4','F5','T']

In [ ]:
_err1 = list()
_result1 = list()
for i in range(0,len(Model1)) :
    _result = get_reculsive_error(_Model=Model1[i],_n_model = 1,std_robot='T',_obs_list=_obs_list,pred_list=pred_list,robots=['H','F1','F2','F3','F4','F5','T'],data_test = data_test,_n_samples=_n_samples,_n_hist=_n_hist,std_coordinates=std_coordinates)
    _error, _pred_result = cal_accuracy_tmp(data=_result,model_n = 1,data_test=data_test,std_coordinates=std_coordinates, _n_samples=_n_samples)
    _err1.append(_error)
    _result1.append(_pred_result)

### Model2

In [6]:
std_coordinates = 310
std_orientation = 180

_obs_list = [['_F5_x','_F5_y','_F4_x','_F4_y'],['_F4_x','_F4_y','_F3_x','_F3_y'],
            ['_F3_x','_F3_y','_F2_x','_F2_y'],['_F2_x','_F2_y','_F1_x','_F1_y']]
pred_list = [['_F3_x','_F3_y','_F3_o','_F4_o','_F5_o'],['_F2_x','_F2_y','_F2_o','_F3_o','_F4_o'],
            ['_F1_x','_F1_y','_F1_o','_F2_o','_F3_o'],['_H_x','_H_y','_H_o','_F1_o','_F2_o']]
robots = ['H','F1','F2','F3','F4','F5','T']

#_obs_list = [['_F3_x','_F3_y','_T_o','_F2_x','_F2_y'],['_F2_x','_F2_y','_F3_o','_F1_x','_F1_y']]
#pred_list = [['_F1_x','_F1_y','_F1_o','_F2_o','_F3_o'],['_H_x','_H_y','_H_o','_F1_o','_F2_o']]
#robots = ['H','F1','F2','F3','T']

In [7]:
_err2 = list()
_result2 = list()
for i in range(0,len(Model2)) :
    _result = get_reculsive_error(_Model=Model2[i],_n_model = 2,std_robot='T',
                                  timedifference=[1,1,0,0],timeshift=1,
#                                  timedifference=[0,0,0,0],timeshift=0,
                                  _obs_list=_obs_list, pred_list=pred_list,
                                  robots=['H','F1','F2','F3','F4','F5','T'],
                                  data_test = data_test,_n_samples=_n_samples,
                                  _n_hist=_n_hist,std_coordinates=std_coordinates)
    _error, _pred_result = cal_accuracy_tmp(data=_result,model_n = 2,data_test=data_test,std_coordinates=std_coordinates, _n_samples=_n_samples)
    _err2.append(_error)
    _result2.append(_pred_result)

History : [['0_F5_x', '0_F5_y', '0_F4_x', '0_F4_y'], ['0_F3_x', '0_F3_y', '0_F3_o', '0_F4_o', '0_F5_o'], []]
History : [['1_F5_x', '1_F5_y', '1_F4_x', '1_F4_y'], ['1_F3_x', '1_F3_y', '1_F3_o', '1_F4_o', '1_F5_o'], []]
History : [['2_F5_x', '2_F5_y', '2_F4_x', '2_F4_y'], ['2_F3_x', '2_F3_y', '2_F3_o', '2_F4_o', '2_F5_o'], []]
History : [['3_F5_x', '3_F5_y', '3_F4_x', '3_F4_y'], ['3_F3_x', '3_F3_y', '3_F3_o', '3_F4_o', '3_F5_o'], []]
History : [['4_F5_x', '4_F5_y', '4_F4_x', '4_F4_y'], ['4_F3_x', '4_F3_y', '4_F3_o', '4_F4_o', '4_F5_o'], []]
History : [['5_F5_x', '5_F5_y', '5_F4_x', '5_F4_y'], ['5_F3_x', '5_F3_y', '5_F3_o', '5_F4_o', '5_F5_o'], []]
History : [['6_F5_x', '6_F5_y', '6_F4_x', '6_F4_y'], ['6_F3_x', '6_F3_y', '6_F3_o', '6_F4_o', '6_F5_o'], []]
History : [['7_F5_x', '7_F5_y', '7_F4_x', '7_F4_y'], ['7_F3_x', '7_F3_y', '7_F3_o', '7_F4_o', '7_F5_o'], []]
History : [['8_F5_x', '8_F5_y', '8_F4_x', '8_F4_y'], ['8_F3_x', '8_F3_y', '8_F3_o', '8_F4_o', '8_F5_o'], []]
History : [['9_F5_x

History : [['11_F5_x', '11_F5_y', '11_F4_x', '11_F4_y'], [], ['11_F3_x', '11_F3_y', '11_F3_o', '11_F4_o', '11_F5_o']]
History : [['12_F5_x', '12_F5_y', '12_F4_x', '12_F4_y'], [], ['12_F3_x', '12_F3_y', '12_F3_o', '12_F4_o', '12_F5_o']]
History : [['13_F5_x', '13_F5_y', '13_F4_x', '13_F4_y'], [], ['13_F3_x', '13_F3_y', '13_F3_o', '13_F4_o', '13_F5_o']]
History : [['14_F5_x', '14_F5_y', '14_F4_x', '14_F4_y'], [], ['14_F3_x', '14_F3_y', '14_F3_o', '14_F4_o', '14_F5_o']]
History : [['15_F5_x', '15_F5_y', '15_F4_x', '15_F4_y'], [], ['15_F3_x', '15_F3_y', '15_F3_o', '15_F4_o', '15_F5_o']]
Observed : ['17_F5_x', '17_F5_y', '16_F4_x', '16_F4_y', '18_F5_x', '18_F5_y', '17_F4_x', '17_F4_y']
Label : [['16_F3_x', '16_F3_y'], ['16_F3_o', '16_F4_o', '16_F5_o', '17_F5_o']]
History : [['7_F5_x', '7_F5_y', '7_F4_x', '7_F4_y'], ['7_F3_x', '7_F3_y', '7_F3_o', '7_F4_o', '7_F5_o'], []]
History : [['8_F5_x', '8_F5_y', '8_F4_x', '8_F4_y'], ['8_F3_x', '8_F3_y', '8_F3_o', '8_F4_o', '8_F5_o'], []]
History : [['

Label : [['22_F3_x', '22_F3_y'], ['22_F3_o', '22_F4_o', '22_F5_o', '23_F5_o']]
History : [['13_F5_x', '13_F5_y', '13_F4_x', '13_F4_y'], [], ['13_F3_x', '13_F3_y', '13_F3_o', '13_F4_o', '13_F5_o']]
History : [['14_F5_x', '14_F5_y', '14_F4_x', '14_F4_y'], [], ['14_F3_x', '14_F3_y', '14_F3_o', '14_F4_o', '14_F5_o']]
History : [['15_F5_x', '15_F5_y', '15_F4_x', '15_F4_y'], [], ['15_F3_x', '15_F3_y', '15_F3_o', '15_F4_o', '15_F5_o']]
History : [['16_F5_x', '16_F5_y', '16_F4_x', '16_F4_y'], [], ['16_F3_x', '16_F3_y', '16_F3_o', '16_F4_o', '16_F5_o']]
History : [['17_F5_x', '17_F5_y', '17_F4_x', '17_F4_y'], [], ['17_F3_x', '17_F3_y', '17_F3_o', '17_F4_o', '17_F5_o']]
History : [['18_F5_x', '18_F5_y', '18_F4_x', '18_F4_y'], [], ['18_F3_x', '18_F3_y', '18_F3_o', '18_F4_o', '18_F5_o']]
History : [['19_F5_x', '19_F5_y', '19_F4_x', '19_F4_y'], [], ['19_F3_x', '19_F3_y', '19_F3_o', '19_F4_o', '19_F5_o']]
History : [['20_F5_x', '20_F5_y', '20_F4_x', '20_F4_y'], [], ['20_F3_x', '20_F3_y', '20_F3_o', 

Label : [['15_F2_x', '15_F2_y'], ['15_F2_o', '15_F3_o', '15_F4_o', '16_F4_o']]
History : [['6_F4_x', '6_F4_y', '6_F3_x', '6_F3_y'], ['6_F2_x', '6_F2_y', '6_F2_o', '6_F3_o', '6_F4_o'], []]
History : [['7_F4_x', '7_F4_y', '7_F3_x', '7_F3_y'], ['7_F2_x', '7_F2_y', '7_F2_o', '7_F3_o', '7_F4_o'], []]
History : [['8_F4_x', '8_F4_y', '8_F3_x', '8_F3_y'], ['8_F2_x', '8_F2_y', '8_F2_o', '8_F3_o', '8_F4_o'], []]
History : [['9_F4_x', '9_F4_y', '9_F3_x', '9_F3_y'], ['9_F2_x', '9_F2_y', '9_F2_o', '9_F3_o', '9_F4_o'], []]
History : [['10_F4_x', '10_F4_y', '10_F3_x', '10_F3_y'], [], ['10_F2_x', '10_F2_y', '10_F2_o', '10_F3_o', '10_F4_o']]
History : [['11_F4_x', '11_F4_y', '11_F3_x', '11_F3_y'], [], ['11_F2_x', '11_F2_y', '11_F2_o', '11_F3_o', '11_F4_o']]
History : [['12_F4_x', '12_F4_y', '12_F3_x', '12_F3_y'], [], ['12_F2_x', '12_F2_y', '12_F2_o', '12_F3_o', '12_F4_o']]
History : [['13_F4_x', '13_F4_y', '13_F3_x', '13_F3_y'], [], ['13_F2_x', '13_F2_y', '13_F2_o', '13_F3_o', '13_F4_o']]
History : [['

History : [['13_F4_x', '13_F4_y', '13_F3_x', '13_F3_y'], [], ['13_F2_x', '13_F2_y', '13_F2_o', '13_F3_o', '13_F4_o']]
History : [['14_F4_x', '14_F4_y', '14_F3_x', '14_F3_y'], [], ['14_F2_x', '14_F2_y', '14_F2_o', '14_F3_o', '14_F4_o']]
History : [['15_F4_x', '15_F4_y', '15_F3_x', '15_F3_y'], [], ['15_F2_x', '15_F2_y', '15_F2_o', '15_F3_o', '15_F4_o']]
History : [['16_F4_x', '16_F4_y', '16_F3_x', '16_F3_y'], [], ['16_F2_x', '16_F2_y', '16_F2_o', '16_F3_o', '16_F4_o']]
History : [['17_F4_x', '17_F4_y', '17_F3_x', '17_F3_y'], [], ['17_F2_x', '17_F2_y', '17_F2_o', '17_F3_o', '17_F4_o']]
History : [['18_F4_x', '18_F4_y', '18_F3_x', '18_F3_y'], [], ['18_F2_x', '18_F2_y', '18_F2_o', '18_F3_o', '18_F4_o']]
History : [['19_F4_x', '19_F4_y', '19_F3_x', '19_F3_y'], [], ['19_F2_x', '19_F2_y', '19_F2_o', '19_F3_o', '19_F4_o']]
History : [['20_F4_x', '20_F4_y', '20_F3_x', '20_F3_y'], [], ['20_F2_x', '20_F2_y', '20_F2_o', '20_F3_o', '20_F4_o']]
History : [['21_F4_x', '21_F4_y', '21_F3_x', '21_F3_y'],

...

Label : [['13_F2_x', '13_F2_y'], ['13_F2_o', '13_F3_o', '13_F4_o', '14_F4_o']]
History : [['4_F4_x', '4_F4_y', '4_F3_x', '4_F3_y'], ['4_F2_x', '4_F2_y', '4_F2_o', '4_F3_o', '4_F4_o'], []]
History : [['5_F4_x', '5_F4_y', '5_F3_x', '5_F3_y'], ['5_F2_x', '5_F2_y', '5_F2_o', '5_F3_o', '5_F4_o'], []]
History : [['6_F4_x', '6_F4_y', '6_F3_x', '6_F3_y'], ['6_F2_x', '6_F2_y', '6_F2_o', '6_F3_o', '6_F4_o'], []]
History : [['7_F4_x', '7_F4_y', '7_F3_x', '7_F3_y'], ['7_F2_x', '7_F2_y', '7_F2_o', '7_F3_o', '7_F4_o'], []]
History : [['8_F4_x', '8_F4_y', '8_F3_x', '8_F3_y'], ['8_F2_x', '8_F2_y', '8_F2_o', '8_F3_o', '8_F4_o'], []]
History : [['9_F4_x', '9_F4_y', '9_F3_x', '9_F3_y'], ['9_F2_x', '9_F2_y', '9_F2_o', '9_F3_o', '9_F4_o'], []]
History : [['10_F4_x', '10_F4_y', '10_F3_x', '10_F3_y'], [], ['10_F2_x', '10_F2_y', '10_F2_o', '10_F3_o', '10_F4_o']]
History : [['11_F4_x', '11_F4_y', '11_F3_x', '11_F3_y'], [], ['11_F2_x', '11_F2_y', '11_F2_o', '11_F3_o', '11_F4_o']]
History : [['12_F4_x', '12_F4_y

Label : [['20_F2_x', '20_F2_y'], ['20_F2_o', '20_F3_o', '20_F4_o', '21_F4_o']]
History : [['11_F4_x', '11_F4_y', '11_F3_x', '11_F3_y'], [], ['11_F2_x', '11_F2_y', '11_F2_o', '11_F3_o', '11_F4_o']]
History : [['12_F4_x', '12_F4_y', '12_F3_x', '12_F3_y'], [], ['12_F2_x', '12_F2_y', '12_F2_o', '12_F3_o', '12_F4_o']]
History : [['13_F4_x', '13_F4_y', '13_F3_x', '13_F3_y'], [], ['13_F2_x', '13_F2_y', '13_F2_o', '13_F3_o', '13_F4_o']]
History : [['14_F4_x', '14_F4_y', '14_F3_x', '14_F3_y'], [], ['14_F2_x', '14_F2_y', '14_F2_o', '14_F3_o', '14_F4_o']]
History : [['15_F4_x', '15_F4_y', '15_F3_x', '15_F3_y'], [], ['15_F2_x', '15_F2_y', '15_F2_o', '15_F3_o', '15_F4_o']]
History : [['16_F4_x', '16_F4_y', '16_F3_x', '16_F3_y'], [], ['16_F2_x', '16_F2_y', '16_F2_o', '16_F3_o', '16_F4_o']]
History : [['17_F4_x', '17_F4_y', '17_F3_x', '17_F3_y'], [], ['17_F2_x', '17_F2_y', '17_F2_o', '17_F3_o', '17_F4_o']]
History : [['18_F4_x', '18_F4_y', '18_F3_x', '18_F3_y'], [], ['18_F2_x', '18_F2_y', '18_F2_o', 

Label : [['14_F1_x', '14_F1_y'], ['14_F1_o', '14_F2_o', '14_F3_o', '15_F3_o']]
History : [['5_F3_x', '5_F3_y', '5_F2_x', '5_F2_y'], ['5_F1_x', '5_F1_y', '5_F1_o', '5_F2_o', '5_F3_o'], []]
History : [['6_F3_x', '6_F3_y', '6_F2_x', '6_F2_y'], ['6_F1_x', '6_F1_y', '6_F1_o', '6_F2_o', '6_F3_o'], []]
History : [['7_F3_x', '7_F3_y', '7_F2_x', '7_F2_y'], ['7_F1_x', '7_F1_y', '7_F1_o', '7_F2_o', '7_F3_o'], []]
History : [['8_F3_x', '8_F3_y', '8_F2_x', '8_F2_y'], ['8_F1_x', '8_F1_y', '8_F1_o', '8_F2_o', '8_F3_o'], []]
History : [['9_F3_x', '9_F3_y', '9_F2_x', '9_F2_y'], ['9_F1_x', '9_F1_y', '9_F1_o', '9_F2_o', '9_F3_o'], []]
History : [['10_F3_x', '10_F3_y', '10_F2_x', '10_F2_y'], [], ['10_F1_x', '10_F1_y', '10_F1_o', '10_F2_o', '10_F3_o']]
History : [['11_F3_x', '11_F3_y', '11_F2_x', '11_F2_y'], [], ['11_F1_x', '11_F1_y', '11_F1_o', '11_F2_o', '11_F3_o']]
History : [['12_F3_x', '12_F3_y', '12_F2_x', '12_F2_y'], [], ['12_F1_x', '12_F1_y', '12_F1_o', '12_F2_o', '12_F3_o']]
History : [['13_F3_x',

Label : [['21_F1_x', '21_F1_y'], ['21_F1_o', '21_F2_o', '21_F3_o', '22_F3_o']]
History : [['0_F2_x', '0_F2_y', '0_F1_x', '0_F1_y'], ['0_H_x', '0_H_y', '0_H_o', '0_F1_o', '0_F2_o'], []]
History : [['1_F2_x', '1_F2_y', '1_F1_x', '1_F1_y'], ['1_H_x', '1_H_y', '1_H_o', '1_F1_o', '1_F2_o'], []]
History : [['2_F2_x', '2_F2_y', '2_F1_x', '2_F1_y'], ['2_H_x', '2_H_y', '2_H_o', '2_F1_o', '2_F2_o'], []]
History : [['3_F2_x', '3_F2_y', '3_F1_x', '3_F1_y'], ['3_H_x', '3_H_y', '3_H_o', '3_F1_o', '3_F2_o'], []]
History : [['4_F2_x', '4_F2_y', '4_F1_x', '4_F1_y'], ['4_H_x', '4_H_y', '4_H_o', '4_F1_o', '4_F2_o'], []]
History : [['5_F2_x', '5_F2_y', '5_F1_x', '5_F1_y'], ['5_H_x', '5_H_y', '5_H_o', '5_F1_o', '5_F2_o'], []]
History : [['6_F2_x', '6_F2_y', '6_F1_x', '6_F1_y'], ['6_H_x', '6_H_y', '6_H_o', '6_F1_o', '6_F2_o'], []]
History : [['7_F2_x', '7_F2_y', '7_F1_x', '7_F1_y'], ['7_H_x', '7_H_y', '7_H_o', '7_F1_o', '7_F2_o'], []]
History : [['8_F2_x', '8_F2_y', '8_F1_x', '8_F1_y'], ['8_H_x', '8_H_y', '

Label : [['16_H_x', '16_H_y'], ['16_H_o', '16_F1_o', '16_F2_o', '17_F2_o']]
History : [['7_F2_x', '7_F2_y', '7_F1_x', '7_F1_y'], ['7_H_x', '7_H_y', '7_H_o', '7_F1_o', '7_F2_o'], []]
History : [['8_F2_x', '8_F2_y', '8_F1_x', '8_F1_y'], ['8_H_x', '8_H_y', '8_H_o', '8_F1_o', '8_F2_o'], []]
History : [['9_F2_x', '9_F2_y', '9_F1_x', '9_F1_y'], ['9_H_x', '9_H_y', '9_H_o', '9_F1_o', '9_F2_o'], []]
History : [['10_F2_x', '10_F2_y', '10_F1_x', '10_F1_y'], [], ['10_H_x', '10_H_y', '10_H_o', '10_F1_o', '10_F2_o']]
History : [['11_F2_x', '11_F2_y', '11_F1_x', '11_F1_y'], [], ['11_H_x', '11_H_y', '11_H_o', '11_F1_o', '11_F2_o']]
History : [['12_F2_x', '12_F2_y', '12_F1_x', '12_F1_y'], [], ['12_H_x', '12_H_y', '12_H_o', '12_F1_o', '12_F2_o']]
History : [['13_F2_x', '13_F2_y', '13_F1_x', '13_F1_y'], [], ['13_H_x', '13_H_y', '13_H_o', '13_F1_o', '13_F2_o']]
History : [['14_F2_x', '14_F2_y', '14_F1_x', '14_F1_y'], [], ['14_H_x', '14_H_y', '14_H_o', '14_F1_o', '14_F2_o']]
History : [['15_F2_x', '15_F2_y

### Model3

In [ ]:
std_coordinates = 400
std_orientation = 180

_obs_list = [['_F5_x','_F5_y','_F4_x','_F4_y','_F3_x','_F3_y'],
            ['_F4_x','_F4_y','_F3_x','_F3_y','_F2_x','_F2_y'],
            ['_F3_x','_F3_y','_F2_x','_F2_y','_F1_x','_F1_y']]
pred_list = [['_F2_x','_F2_y','_F2_o','_F3_o','_F4_o','_F5_o'],
            ['_F1_x','_F1_y','_F1_o','_F2_o','_F3_o','_F4_o'],
            ['_H_x','_H_y','_H_o','_F1_o','_F2_o','_F3_o']]
#_obs_list = [['_F3_x','_F3_y','_T_o','_F2_x','_F2_y','_F1_x','_F1_y']]
#pred_list = [['_H_x','_H_y','_H_o','_F1_o','_F2_o','_F3_o']]


In [ ]:
_err3 = list()
_result3 = list()
for i in range(0,len(Model3)) :
    _result = get_reculsive_error(_Model=Model3[i],_n_model = 3,std_robot='T',
                                  _obs_list=_obs_list, pred_list=pred_list,
                                  timedifference=[2,2,1,1,0,0],
                                  timeshift=2,
#                                  timedifference=[0,0,0,0,0,0],
 #                                 timeshift=0,
                                  robots=['H','F1','F2','F3','F4','F5','T'],
                                  data_test = data_test,_n_samples=_n_samples,
                                  _n_hist=_n_hist,std_coordinates=std_coordinates)
    _error, _pred_result = cal_accuracy_tmp(data=_result,model_n = 3,data_test=data_test,std_coordinates=std_coordinates, _n_samples=_n_samples)
    _err3.append(_error)
    _result3.append(_pred_result)


# Additional Models

### Model3-2 : Use only F1 data

In [6]:
std_coordinates = 400
std_orientation = 180

_obs_list = [['_F3_x','_F3_y'],
            ['_F2_x','_F2_y'],
            ['_F1_x','_F1_y']]
pred_list = [['_F2_x','_F2_y','_F2_o','_F3_o'],
            ['_F1_x','_F1_y','_F1_o','_F2_o'],
            ['_H_x','_H_y','_H_o','_F1_o']]

#_obs_list = [['_F3_x','_F3_y','_T_o','_F2_x','_F2_y','_F1_x','_F1_y']]
#pred_list = [['_H_x','_H_y','_H_o','_F1_o','_F2_o','_F3_o']]

In [7]:
_err3_2 = list()
_result3_2 = list()
for i in range(0,len(Model3_2)) :
    _result = get_reculsive_error(_Model=Model3_2[i],_n_model = 3,std_robot='T',
                                  _obs_list=_obs_list, pred_list=pred_list,
                                  timedifference=[0,0,0,0,0,0],timeshift=0,
                                  robots=['H','F1','F2','F3','F4','F5','T'],
                                  data_test = data_test,_n_samples=_n_samples,
                                  _n_hist=_n_hist,std_coordinates=std_coordinates)
    _error, _pred_result = cal_accuracy_tmp(data=_result,model_n = 3,data_test=data_test,std_coordinates=std_coordinates, _n_samples=_n_samples)
    _err3_2.append(_error)
    _result3_2.append(_pred_result)


History : [['0_F3_x', '0_F3_y'], ['0_F2_x', '0_F2_y', '0_F2_o', '0_F3_o'], []]
History : [['1_F3_x', '1_F3_y'], ['1_F2_x', '1_F2_y', '1_F2_o', '1_F3_o'], []]
History : [['2_F3_x', '2_F3_y'], ['2_F2_x', '2_F2_y', '2_F2_o', '2_F3_o'], []]
History : [['3_F3_x', '3_F3_y'], ['3_F2_x', '3_F2_y', '3_F2_o', '3_F3_o'], []]
History : [['4_F3_x', '4_F3_y'], ['4_F2_x', '4_F2_y', '4_F2_o', '4_F3_o'], []]
History : [['5_F3_x', '5_F3_y'], ['5_F2_x', '5_F2_y', '5_F2_o', '5_F3_o'], []]
History : [['6_F3_x', '6_F3_y'], ['6_F2_x', '6_F2_y', '6_F2_o', '6_F3_o'], []]
History : [['7_F3_x', '7_F3_y'], ['7_F2_x', '7_F2_y', '7_F2_o', '7_F3_o'], []]
History : [['8_F3_x', '8_F3_y'], ['8_F2_x', '8_F2_y', '8_F2_o', '8_F3_o'], []]
History : [['9_F3_x', '9_F3_y'], ['9_F2_x', '9_F2_y', '9_F2_o', '9_F3_o'], []]
Observed : ['10_F3_x', '10_F3_y', '11_F3_x', '11_F3_y']
Label : [['10_F2_x', '10_F2_y'], ['10_F2_o', '10_F3_o', '10_F4_o', '10_F5_o', '11_F5_o']]
History : [['1_F3_x', '1_F3_y'], ['1_F2_x', '1_F2_y', '1_F2_o', 

Label : [['18_F2_x', '18_F2_y'], ['18_F2_o', '18_F3_o', '18_F4_o', '18_F5_o', '19_F5_o']]
History : [['9_F3_x', '9_F3_y'], ['9_F2_x', '9_F2_y', '9_F2_o', '9_F3_o'], []]
History : [['10_F3_x', '10_F3_y'], [], ['10_F2_x', '10_F2_y', '10_F2_o', '10_F3_o']]
History : [['11_F3_x', '11_F3_y'], [], ['11_F2_x', '11_F2_y', '11_F2_o', '11_F3_o']]
History : [['12_F3_x', '12_F3_y'], [], ['12_F2_x', '12_F2_y', '12_F2_o', '12_F3_o']]
History : [['13_F3_x', '13_F3_y'], [], ['13_F2_x', '13_F2_y', '13_F2_o', '13_F3_o']]
History : [['14_F3_x', '14_F3_y'], [], ['14_F2_x', '14_F2_y', '14_F2_o', '14_F3_o']]
History : [['15_F3_x', '15_F3_y'], [], ['15_F2_x', '15_F2_y', '15_F2_o', '15_F3_o']]
History : [['16_F3_x', '16_F3_y'], [], ['16_F2_x', '16_F2_y', '16_F2_o', '16_F3_o']]
History : [['17_F3_x', '17_F3_y'], [], ['17_F2_x', '17_F2_y', '17_F2_o', '17_F3_o']]
History : [['18_F3_x', '18_F3_y'], [], ['18_F2_x', '18_F2_y', '18_F2_o', '18_F3_o']]
Observed : ['19_F3_x', '19_F3_y', '20_F3_x', '20_F3_y']
Label : [[

History : [['3_F2_x', '3_F2_y'], ['3_F1_x', '3_F1_y', '3_F1_o', '3_F2_o'], []]
History : [['4_F2_x', '4_F2_y'], ['4_F1_x', '4_F1_y', '4_F1_o', '4_F2_o'], []]
History : [['5_F2_x', '5_F2_y'], ['5_F1_x', '5_F1_y', '5_F1_o', '5_F2_o'], []]
History : [['6_F2_x', '6_F2_y'], ['6_F1_x', '6_F1_y', '6_F1_o', '6_F2_o'], []]
History : [['7_F2_x', '7_F2_y'], ['7_F1_x', '7_F1_y', '7_F1_o', '7_F2_o'], []]
History : [['8_F2_x', '8_F2_y'], ['8_F1_x', '8_F1_y', '8_F1_o', '8_F2_o'], []]
History : [['9_F2_x', '9_F2_y'], ['9_F1_x', '9_F1_y', '9_F1_o', '9_F2_o'], []]
History : [['10_F2_x', '10_F2_y'], [], ['10_F1_x', '10_F1_y', '10_F1_o', '10_F2_o']]
History : [['11_F2_x', '11_F2_y'], [], ['11_F1_x', '11_F1_y', '11_F1_o', '11_F2_o']]
History : [['12_F2_x', '12_F2_y'], [], ['12_F1_x', '12_F1_y', '12_F1_o', '12_F2_o']]
Observed : ['13_F2_x', '13_F2_y', '14_F2_x', '14_F2_y']
Label : [['13_F1_x', '13_F1_y'], ['13_F1_o', '13_F2_o', '13_F3_o', '13_F4_o', '14_F4_o']]
History : [['4_F2_x', '4_F2_y'], ['4_F1_x', '4

Label : [['21_F1_x', '21_F1_y'], ['21_F1_o', '21_F2_o', '21_F3_o', '21_F4_o', '22_F4_o']]
History : [['12_F2_x', '12_F2_y'], [], ['12_F1_x', '12_F1_y', '12_F1_o', '12_F2_o']]
History : [['13_F2_x', '13_F2_y'], [], ['13_F1_x', '13_F1_y', '13_F1_o', '13_F2_o']]
History : [['14_F2_x', '14_F2_y'], [], ['14_F1_x', '14_F1_y', '14_F1_o', '14_F2_o']]
History : [['15_F2_x', '15_F2_y'], [], ['15_F1_x', '15_F1_y', '15_F1_o', '15_F2_o']]
History : [['16_F2_x', '16_F2_y'], [], ['16_F1_x', '16_F1_y', '16_F1_o', '16_F2_o']]
History : [['17_F2_x', '17_F2_y'], [], ['17_F1_x', '17_F1_y', '17_F1_o', '17_F2_o']]
History : [['18_F2_x', '18_F2_y'], [], ['18_F1_x', '18_F1_y', '18_F1_o', '18_F2_o']]
History : [['19_F2_x', '19_F2_y'], [], ['19_F1_x', '19_F1_y', '19_F1_o', '19_F2_o']]
History : [['20_F2_x', '20_F2_y'], [], ['20_F1_x', '20_F1_y', '20_F1_o', '20_F2_o']]
History : [['21_F2_x', '21_F2_y'], [], ['21_F1_x', '21_F1_y', '21_F1_o', '21_F2_o']]
Observed : ['22_F2_x', '22_F2_y', '23_F2_x', '23_F2_y']
Labe

Label : [['16_H_x', '16_H_y'], ['16_H_o', '16_F1_o', '16_F2_o', '16_F3_o', '17_F3_o']]
History : [['7_F1_x', '7_F1_y'], ['7_H_x', '7_H_y', '7_H_o', '7_F1_o'], []]
History : [['8_F1_x', '8_F1_y'], ['8_H_x', '8_H_y', '8_H_o', '8_F1_o'], []]
History : [['9_F1_x', '9_F1_y'], ['9_H_x', '9_H_y', '9_H_o', '9_F1_o'], []]
History : [['10_F1_x', '10_F1_y'], [], ['10_H_x', '10_H_y', '10_H_o', '10_F1_o']]
History : [['11_F1_x', '11_F1_y'], [], ['11_H_x', '11_H_y', '11_H_o', '11_F1_o']]
History : [['12_F1_x', '12_F1_y'], [], ['12_H_x', '12_H_y', '12_H_o', '12_F1_o']]
History : [['13_F1_x', '13_F1_y'], [], ['13_H_x', '13_H_y', '13_H_o', '13_F1_o']]
History : [['14_F1_x', '14_F1_y'], [], ['14_H_x', '14_H_y', '14_H_o', '14_F1_o']]
History : [['15_F1_x', '15_F1_y'], [], ['15_H_x', '15_H_y', '15_H_o', '15_F1_o']]
History : [['16_F1_x', '16_F1_y'], [], ['16_H_x', '16_H_y', '16_H_o', '16_F1_o']]
Observed : ['17_F1_x', '17_F1_y', '18_F1_x', '18_F1_y']
Label : [['17_H_x', '17_H_y'], ['17_H_o', '17_F1_o', '1

...

Label : [['12_F2_x', '12_F2_y'], ['12_F2_o', '12_F3_o', '12_F4_o', '12_F5_o', '13_F5_o']]
History : [['3_F3_x', '3_F3_y'], ['3_F2_x', '3_F2_y', '3_F2_o', '3_F3_o'], []]
History : [['4_F3_x', '4_F3_y'], ['4_F2_x', '4_F2_y', '4_F2_o', '4_F3_o'], []]
History : [['5_F3_x', '5_F3_y'], ['5_F2_x', '5_F2_y', '5_F2_o', '5_F3_o'], []]
History : [['6_F3_x', '6_F3_y'], ['6_F2_x', '6_F2_y', '6_F2_o', '6_F3_o'], []]
History : [['7_F3_x', '7_F3_y'], ['7_F2_x', '7_F2_y', '7_F2_o', '7_F3_o'], []]
History : [['8_F3_x', '8_F3_y'], ['8_F2_x', '8_F2_y', '8_F2_o', '8_F3_o'], []]
History : [['9_F3_x', '9_F3_y'], ['9_F2_x', '9_F2_y', '9_F2_o', '9_F3_o'], []]
History : [['10_F3_x', '10_F3_y'], [], ['10_F2_x', '10_F2_y', '10_F2_o', '10_F3_o']]
History : [['11_F3_x', '11_F3_y'], [], ['11_F2_x', '11_F2_y', '11_F2_o', '11_F3_o']]
History : [['12_F3_x', '12_F3_y'], [], ['12_F2_x', '12_F2_y', '12_F2_o', '12_F3_o']]
Observed : ['13_F3_x', '13_F3_y', '14_F3_x', '14_F3_y']
Label : [['13_F2_x', '13_F2_y'], ['13_F2_o', '

Label : [['21_F2_x', '21_F2_y'], ['21_F2_o', '21_F3_o', '21_F4_o', '21_F5_o', '22_F5_o']]
History : [['12_F3_x', '12_F3_y'], [], ['12_F2_x', '12_F2_y', '12_F2_o', '12_F3_o']]
History : [['13_F3_x', '13_F3_y'], [], ['13_F2_x', '13_F2_y', '13_F2_o', '13_F3_o']]
History : [['14_F3_x', '14_F3_y'], [], ['14_F2_x', '14_F2_y', '14_F2_o', '14_F3_o']]
History : [['15_F3_x', '15_F3_y'], [], ['15_F2_x', '15_F2_y', '15_F2_o', '15_F3_o']]
History : [['16_F3_x', '16_F3_y'], [], ['16_F2_x', '16_F2_y', '16_F2_o', '16_F3_o']]
History : [['17_F3_x', '17_F3_y'], [], ['17_F2_x', '17_F2_y', '17_F2_o', '17_F3_o']]
History : [['18_F3_x', '18_F3_y'], [], ['18_F2_x', '18_F2_y', '18_F2_o', '18_F3_o']]
History : [['19_F3_x', '19_F3_y'], [], ['19_F2_x', '19_F2_y', '19_F2_o', '19_F3_o']]
History : [['20_F3_x', '20_F3_y'], [], ['20_F2_x', '20_F2_y', '20_F2_o', '20_F3_o']]
History : [['21_F3_x', '21_F3_y'], [], ['21_F2_x', '21_F2_y', '21_F2_o', '21_F3_o']]
Observed : ['22_F3_x', '22_F3_y', '23_F3_x', '23_F3_y']
Labe

Label : [['15_F1_x', '15_F1_y'], ['15_F1_o', '15_F2_o', '15_F3_o', '15_F4_o', '16_F4_o']]
History : [['6_F2_x', '6_F2_y'], ['6_F1_x', '6_F1_y', '6_F1_o', '6_F2_o'], []]
History : [['7_F2_x', '7_F2_y'], ['7_F1_x', '7_F1_y', '7_F1_o', '7_F2_o'], []]
History : [['8_F2_x', '8_F2_y'], ['8_F1_x', '8_F1_y', '8_F1_o', '8_F2_o'], []]
History : [['9_F2_x', '9_F2_y'], ['9_F1_x', '9_F1_y', '9_F1_o', '9_F2_o'], []]
History : [['10_F2_x', '10_F2_y'], [], ['10_F1_x', '10_F1_y', '10_F1_o', '10_F2_o']]
History : [['11_F2_x', '11_F2_y'], [], ['11_F1_x', '11_F1_y', '11_F1_o', '11_F2_o']]
History : [['12_F2_x', '12_F2_y'], [], ['12_F1_x', '12_F1_y', '12_F1_o', '12_F2_o']]
History : [['13_F2_x', '13_F2_y'], [], ['13_F1_x', '13_F1_y', '13_F1_o', '13_F2_o']]
History : [['14_F2_x', '14_F2_y'], [], ['14_F1_x', '14_F1_y', '14_F1_o', '14_F2_o']]
History : [['15_F2_x', '15_F2_y'], [], ['15_F1_x', '15_F1_y', '15_F1_o', '15_F2_o']]
Observed : ['16_F2_x', '16_F2_y', '17_F2_x', '17_F2_y']
Label : [['16_F1_x', '16_F1_

History : [['1_F1_x', '1_F1_y'], ['1_H_x', '1_H_y', '1_H_o', '1_F1_o'], []]
History : [['2_F1_x', '2_F1_y'], ['2_H_x', '2_H_y', '2_H_o', '2_F1_o'], []]
History : [['3_F1_x', '3_F1_y'], ['3_H_x', '3_H_y', '3_H_o', '3_F1_o'], []]
History : [['4_F1_x', '4_F1_y'], ['4_H_x', '4_H_y', '4_H_o', '4_F1_o'], []]
History : [['5_F1_x', '5_F1_y'], ['5_H_x', '5_H_y', '5_H_o', '5_F1_o'], []]
History : [['6_F1_x', '6_F1_y'], ['6_H_x', '6_H_y', '6_H_o', '6_F1_o'], []]
History : [['7_F1_x', '7_F1_y'], ['7_H_x', '7_H_y', '7_H_o', '7_F1_o'], []]
History : [['8_F1_x', '8_F1_y'], ['8_H_x', '8_H_y', '8_H_o', '8_F1_o'], []]
History : [['9_F1_x', '9_F1_y'], ['9_H_x', '9_H_y', '9_H_o', '9_F1_o'], []]
History : [['10_F1_x', '10_F1_y'], [], ['10_H_x', '10_H_y', '10_H_o', '10_F1_o']]
Observed : ['11_F1_x', '11_F1_y', '12_F1_x', '12_F1_y']
Label : [['11_H_x', '11_H_y'], ['11_H_o', '11_F1_o', '11_F2_o', '11_F3_o', '12_F3_o']]
History : [['2_F1_x', '2_F1_y'], ['2_H_x', '2_H_y', '2_H_o', '2_F1_o'], []]
History : [['3_

Label : [['19_H_x', '19_H_y'], ['19_H_o', '19_F1_o', '19_F2_o', '19_F3_o', '20_F3_o']]
History : [['10_F1_x', '10_F1_y'], [], ['10_H_x', '10_H_y', '10_H_o', '10_F1_o']]
History : [['11_F1_x', '11_F1_y'], [], ['11_H_x', '11_H_y', '11_H_o', '11_F1_o']]
History : [['12_F1_x', '12_F1_y'], [], ['12_H_x', '12_H_y', '12_H_o', '12_F1_o']]
History : [['13_F1_x', '13_F1_y'], [], ['13_H_x', '13_H_y', '13_H_o', '13_F1_o']]
History : [['14_F1_x', '14_F1_y'], [], ['14_H_x', '14_H_y', '14_H_o', '14_F1_o']]
History : [['15_F1_x', '15_F1_y'], [], ['15_H_x', '15_H_y', '15_H_o', '15_F1_o']]
History : [['16_F1_x', '16_F1_y'], [], ['16_H_x', '16_H_y', '16_H_o', '16_F1_o']]
History : [['17_F1_x', '17_F1_y'], [], ['17_H_x', '17_H_y', '17_H_o', '17_F1_o']]
History : [['18_F1_x', '18_F1_y'], [], ['18_H_x', '18_H_y', '18_H_o', '18_F1_o']]
History : [['19_F1_x', '19_F1_y'], [], ['19_H_x', '19_H_y', '19_H_o', '19_F1_o']]
Observed : ['20_F1_x', '20_F1_y', '21_F1_x', '21_F1_y']
Label : [['20_H_x', '20_H_y'], ['20_H

## Save Results

In [8]:
for i in range(0,len(_result2)) :
#    _err1[i].to_csv('Error_1_{}.csv'.format(i))
    _result2[i].to_csv('Results/Predict_Modified_M2_{}.csv'.format(i+5))    
#    _result3_2[i].to_csv('Results/Predict_Modified_M3_nearestonly_{}.csv'.format(i+5))    

In [ ]:
std = [260,310,400]
for i in range(len(std)) :
    test_data = s_relative_and_std(data = data_test, std_coordinates = std[i],n_time_in_instance=_n_samples,std_robot ='T',other_robots=['H','F1','F3','F2','F4','F5'])
    test_data.to_csv('Results/Answer_m{}.csv'.format(i+1))

## Evaluate Orientation

In [ ]:
def error_orientation(answer='',data1='',start=10,end=25,robot='H') :
    _error = list()
    for i in range(start,end) :
        _o = '{}_{}_o'.format(i,robot)
        _tmp = abs(answer[_o]-data1[_o])*100
        _error.append(np.average(_tmp))
        
    return _error

In [ ]:
_error_o_m1 = error_orientation(answer = test_data, data1 = _pred_result_m1_2)
_error_o_m33 = error_orientation(answer = test_data, data1 = _pred_result_m3_3)


plt.plot(_error_o_m1[:11],label='m1_H_o')
plt.plot(_error_o_m33[:13],label='m33_H_o')
plt.legend()
plt.show()

In [ ]:
_error_o_m1 = error_orientation(answer = test_data, data1 = _pred_result_m1,robot='F1')
_error_o_m33 = error_orientation(answer = test_data, data1 = _pred_result_m3_3,robot='F1')
print(_error_o_m1)
print(_error_o_m33)

plt.plot(_error_o_m1[:11],label='m1_F1_o')
plt.plot(_error_o_m33[:11],label='m33_F1_o')
plt.legend()
plt.show()

In [ ]:
_error_o_m1 = error_orientation(answer = test_data, data1 = _pred_result_m1,robot='F2')
_error_o_m33 = error_orientation(answer = test_data, data1 = _pred_result_m3_3,robot='F2')
print(_error_o_m1)
print(_error_o_m33)

plt.plot(_error_o_m1[:12],label='m1_F2_o')
plt.plot(_error_o_m33[:12],label='m33_F2_o')
plt.legend()
plt.show()